In [14]:
import numpy as np
import pandas as pd
import re
import random
from bs4 import BeautifulSoup as bs
import requests
import datetime
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics.pairwise import cosine_similarity, haversine_distances
from math import radians
from sklearn.neighbors import DistanceMetric
global dist 
dist = DistanceMetric.get_metric('haversine')
import ipywidgets as widgets
from IPython.display import display
import warnings
warnings.filterwarnings("ignore")


In [212]:
df = pd.read_csv("Final Dataset before Normalisation.csv", index_col=[0])
df

,Name,Gender,DOB,Name Of Area,Pincode,Latitude,Longitude,Flexibility,Body Composition,Composition Percentage,Interactiveness,Workout Frequency,Workout Mode Preferred,is_sports_enthusiast,like_to_be_mentored,like_to_mentor,fitness_goal_to_be achieved,addons in session,Height_in_cms,Age
0,Amoli Veer,Female,1982-03-31,Nana Peth S.O,411002,18.51009,73.85672,4,Essential Fat,12,4,Only Weekends,Outdoor,No,3,3,All,Meditation,166,39
1,Kaia Subramanian,Female,1994-06-09,Mundhva AV S.O,411036,18.53135,73.91478,5,Fitness,22,5,Alternate Days,Outdoor,Maybe,1,3,Improve endurance/conditioning,Meditation,163,26
2,Rushil Vish,Male,1999-02-07,Yerwada S.O,411006,18.56318,73.87742,3,Acceptable,22,3,Monday To Friday,Both,Yes,4,5,Improve athletic skills,Meditation,171,22
3,Atiksh Manohar,Male,1987-07-01,Dapodi S.O,411012,18.57749,73.83138,5,Athletes,10,3,Alternate Days,Both,Maybe,3,4,Lose fat,Meditation,173,33
4,Ved Kapoor,Male,1988-01-28,Hadapsar S.O,411028,18.50453,73.91255,5,Essential Fat,6,3,3 days of week,Indoor,No,1,1,Build muscle,None,161,33
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1382,Neysa Rai,Female,1989-12-10,Mohamadwadi S.O,411060,18.47335,73.90957,2,Fitness,22,4,Any Day of Week,Both,Maybe,2,4,Improve athletic skills,Yoga,170,31
1383,Raunak Kapoor,Male,1984-08-12,Masulkar Colony S.O,411018,18.63374,73.81290,3,Fitness,15,1,Monday To Friday,Outdoor,Yes,2,5,All,None,161,36
1384,Taara Veer,Female,1995-06-06,Masulkar Colony S.O,411018,18.63374,73.81290,5,Fitness,22,1,Only Weekends,Indoor,No,5,4,Build muscle,Both,166,25
1385,Sarthak Rai,Male,1997-09-03,Guruwar Peth S.O,411042,18.50547,73.85951,5,Fitness,15,5,Monday To Friday,Indoor,Maybe,3,2,Improve athletic skills,Yoga,177,23


## Dividing Datasets

In [16]:
def get_both_datasets_for_normalisation(df):
    df1 = df[['Name', 'Gender', 'Flexibility', 'Pincode', 'Body Composition', 'Composition Percentage', 'Interactiveness', 'Workout Frequency', 'Workout Mode Preferred', 'is_sports_enthusiast', 'like_to_be_mentored', 'like_to_mentor', 'fitness_goal_to_be achieved','addons in session','Height_in_cms','Age']]
    df2 =df[['Name', 'Name Of Area', 'Pincode', 'Latitude ','Longitude']]
    return df1,df2

In [17]:
df1,df2 = get_both_datasets_for_normalisation(df)

In [18]:

def get_df1_normalised(df, names):
    df['Gender'] = df["Gender"].astype('category').cat.codes
    df['Body Composition'] = df["Body Composition"].astype('category').cat.codes
    df['Workout Frequency'] = df["Workout Frequency"].astype('category').cat.codes
    df['Workout Mode Preferred'] = df["Workout Mode Preferred"].astype('category').cat.codes
    df['is_sports_enthusiast'] = df["is_sports_enthusiast"].astype('category').cat.codes
    df['fitness_goal_to_be achieved'] = df["fitness_goal_to_be achieved"].astype('category').cat.codes
    df['addons in session'] = df["addons in session"].astype('category').cat.codes
    df[names] = MinMaxScaler().fit_transform(df[names])
    return df

In [19]:
names = ['Flexibility','Composition Percentage','Pincode', 'Interactiveness', 'like_to_be_mentored', 'like_to_mentor', 'Height_in_cms', 'Age']
df1 = get_df1_normalised(df1,names)

In [213]:
df1

,Name,Gender,Flexibility,Pincode,Body Composition,Composition Percentage,Interactiveness,Workout Frequency,Workout Mode Preferred,is_sports_enthusiast,like_to_be_mentored,like_to_mentor,fitness_goal_to_be achieved,addons in session,Height_in_cms,Age
0,Amoli Veer,0,0.75,0.000898,2,0.12,0.75,5,2,1,0.50,0.50,0,1,0.333333,0.956522
1,Kaia Subramanian,0,1.00,0.031447,3,0.32,1.00,1,2,0,0.00,0.50,3,1,0.166667,0.391304
2,Rushil Vish,1,0.50,0.004492,0,0.32,0.50,4,0,2,0.75,1.00,2,1,0.611111,0.217391
3,Atiksh Manohar,1,1.00,0.009883,1,0.08,0.50,1,0,0,0.50,0.75,4,1,0.722222,0.695652
4,Ved Kapoor,1,1.00,0.024259,2,0.00,0.50,0,1,1,0.00,0.00,1,2,0.055556,0.695652
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1382,Neysa Rai,0,0.25,0.053010,3,0.32,0.75,2,0,0,0.25,0.75,2,3,0.555556,0.608696
1383,Raunak Kapoor,1,0.50,0.015274,3,0.18,0.00,4,2,2,0.25,1.00,0,2,0.055556,0.826087
1384,Taara Veer,0,1.00,0.015274,3,0.32,0.00,5,1,1,1.00,0.75,1,0,0.333333,0.347826
1385,Sarthak Rai,1,1.00,0.036837,3,0.18,1.00,4,1,0,0.50,0.25,2,3,0.944444,0.260870


In [21]:
df1_scaled = df1.copy()

## Getting Cosine Similarity Matrix


In [22]:
def getSimilarityMatrix(df1_scaled):
    similarity_cos = cosine_similarity(df1_scaled.drop('Name',axis=1))
    df1_similarity = pd.DataFrame(similarity_cos, columns=df1_scaled['Name'], index=df1_scaled['Name'])
    return df1_similarity

In [214]:
df1_similarity_test = getSimilarityMatrix(df1_scaled)
df1_similarity_test

Name,Amoli Veer,Kaia Subramanian,Rushil Vish,Atiksh Manohar,Ved Kapoor,Vihaan Chauhan,Tanvi Sandeep,Taara Chauhan,Anaisha Veena,Rebecca Chander,...,Rohan Nagpal,Alisha Roy,Sneha Ranga,Ira Naran,Avni Nath,Neysa Rai,Raunak Kapoor,Taara Veer,Sarthak Rai,Kairav Dutta
Name,,,,,,,,,,,,,,,,,,,,,
Amoli Veer,1.000000,0.576278,0.764181,0.361788,0.475178,0.839872,0.400600,0.643023,0.496650,0.363565,...,0.436768,0.575520,0.834565,0.419347,0.693091,0.647972,0.922747,0.931432,0.828498,0.860630
Kaia Subramanian,0.576278,1.000000,0.464381,0.779724,0.774249,0.790255,0.881326,0.807988,0.856773,0.788733,...,0.437792,0.896195,0.739324,0.591120,0.605904,0.799657,0.625648,0.635266,0.780074,0.443571
Rushil Vish,0.764181,0.464381,1.000000,0.642193,0.400436,0.800647,0.333760,0.789015,0.588003,0.572836,...,0.576720,0.350883,0.849133,0.194214,0.447531,0.600946,0.730432,0.781194,0.747965,0.760684
Atiksh Manohar,0.361788,0.779724,0.642193,1.000000,0.605155,0.686441,0.711210,0.839542,0.796664,0.907987,...,0.600453,0.583284,0.704879,0.281498,0.212589,0.714139,0.394981,0.480231,0.690128,0.293672
Ved Kapoor,0.475178,0.774249,0.400436,0.605155,1.000000,0.599109,0.837260,0.685991,0.846541,0.652275,...,0.648820,0.645725,0.510913,0.620360,0.602202,0.768453,0.685620,0.438285,0.727103,0.426002
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Neysa Rai,0.647972,0.799657,0.600946,0.714139,0.768453,0.821575,0.826965,0.827691,0.859290,0.763485,...,0.726873,0.710704,0.694075,0.588808,0.449129,1.000000,0.732233,0.674641,0.919988,0.418134
Raunak Kapoor,0.922747,0.625648,0.730432,0.394981,0.685620,0.846631,0.542939,0.697051,0.646033,0.421986,...,0.493623,0.629614,0.765830,0.573263,0.808455,0.732233,1.000000,0.872380,0.844839,0.857150
Taara Veer,0.931432,0.635266,0.781194,0.480231,0.438285,0.925836,0.423614,0.667283,0.551617,0.422934,...,0.318198,0.676695,0.830865,0.511681,0.674167,0.674641,0.872380,1.000000,0.821598,0.813507


In [24]:
def getUserCorrelation(df1_similarity,name):
    df1_user = df1_similarity.loc[name,:]
    return df1_user

In [215]:
test_user_corr = getUserCorrelation(df1_similarity_test, 'Amoli Veer')


In [216]:
test_user_corr

Name
Amoli Veer          1.000000
Kaia Subramanian    0.576278
Rushil Vish         0.764181
Atiksh Manohar      0.361788
Ved Kapoor          0.475178
                      ...   
Neysa Rai           0.647972
Raunak Kapoor       0.922747
Taara Veer          0.931432
Sarthak Rai         0.828498
Kairav Dutta        0.860630
Name: Amoli Veer, Length: 1387, dtype: float64

## Preprocessing for Haversine Distance similarity

In [26]:
def addLatLongRad(df2):
    df2['Latitude_rad'] = np.radians(df2['Latitude '])
    df2['Longitude_rad'] = np.radians(df2['Longitude'])
    return df2

In [27]:
df2 = addLatLongRad(df2)

In [28]:
df2

,Name,Name Of Area,Pincode,Latitude,Longitude,Latitude_rad,Longitude_rad
0,Amoli Veer,Nana Peth S.O,411002,18.51009,73.85672,0.323062,1.289043
1,Kaia Subramanian,Mundhva AV S.O,411036,18.53135,73.91478,0.323433,1.290056
2,Rushil Vish,Yerwada S.O,411006,18.56318,73.87742,0.323989,1.289404
3,Atiksh Manohar,Dapodi S.O,411012,18.57749,73.83138,0.324238,1.288601
4,Ved Kapoor,Hadapsar S.O,411028,18.50453,73.91255,0.322965,1.290017
...,...,...,...,...,...,...,...
1382,Neysa Rai,Mohamadwadi S.O,411060,18.47335,73.90957,0.322421,1.289965
1383,Raunak Kapoor,Masulkar Colony S.O,411018,18.63374,73.81290,0.325220,1.288278
1384,Taara Veer,Masulkar Colony S.O,411018,18.63374,73.81290,0.325220,1.288278
1385,Sarthak Rai,Guruwar Peth S.O,411042,18.50547,73.85951,0.322981,1.289092


In [217]:
def getDistanceMatrix(df2):
    distance_similarity_df2 = pd.DataFrame(dist.pairwise(df2[['Latitude_rad','Longitude_rad']].to_numpy())*6371,  columns=df2.Name.unique(), index=df2.Name.unique())
    return distance_similarity_df2

In [218]:
test_distance_matrix = getDistanceMatrix(df2)
test_distance_matrix

,Amoli Veer,Kaia Subramanian,Rushil Vish,Atiksh Manohar,Ved Kapoor,Vihaan Chauhan,Tanvi Sandeep,Taara Chauhan,Anaisha Veena,Rebecca Chander,...,Rohan Nagpal,Alisha Roy,Sneha Ranga,Ira Naran,Avni Nath,Neysa Rai,Raunak Kapoor,Taara Veer,Sarthak Rai,Kairav Dutta
Amoli Veer,0.000000,6.562216,6.293802,7.956407,5.919329,1.698209,3.878664,2.904949,5.965892,7.027572,...,2.342975,14.641492,2.973433,11.291795,33.467366,6.910185,14.504325,14.504325,0.591993,11.029489
Kaia Subramanian,6.562216,0.000000,5.295139,10.179151,2.991502,8.189562,3.364122,7.124112,3.279127,7.104329,...,8.420922,18.165245,8.209169,4.763670,35.281101,6.472663,15.650144,15.650144,6.499336,12.648496
Rushil Vish,6.293802,5.295139,0.000000,5.107074,7.499887,7.125896,3.248009,8.659993,7.757147,1.814675,...,6.640876,13.115020,9.167550,8.140429,30.016782,10.548165,10.382359,10.382359,6.689094,7.361470
Atiksh Manohar,7.956407,10.179151,5.107074,0.000000,11.791663,7.631597,7.249081,10.859106,11.999226,3.436508,...,6.627419,8.020077,10.772929,13.199227,25.746387,14.214540,6.550890,6.550890,8.539714,3.073650
Ved Kapoor,5.919329,2.991502,7.499887,11.791663,0.000000,7.603035,4.555992,5.216033,0.289263,9.191280,...,8.195036,19.514164,6.449006,6.916275,37.390595,3.481271,17.797557,17.797557,5.593817,14.572734
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Neysa Rai,6.910185,6.472663,10.548165,14.214540,3.481271,8.302981,7.348445,4.686601,3.194399,12.087887,...,9.201000,21.473476,5.798780,9.997217,39.960902,0.000000,20.540652,20.540652,6.373775,17.175652
Raunak Kapoor,14.504325,15.650144,10.382359,6.550890,17.797557,14.129378,13.286032,17.404263,18.037327,8.618179,...,13.094386,6.245470,17.315909,17.368142,19.634914,20.540652,0.000000,0.000000,15.085411,3.478182
Taara Veer,14.504325,15.650144,10.382359,6.550890,17.797557,14.129378,13.286032,17.404263,18.037327,8.618179,...,13.094386,6.245470,17.315909,17.368142,19.634914,20.540652,0.000000,0.000000,15.085411,3.478182
Sarthak Rai,0.591993,6.499336,6.689094,8.539714,5.593817,2.038662,4.067700,2.319391,5.616184,7.515995,...,2.833156,15.217746,2.491584,11.256827,34.058992,6.373775,15.085411,15.085411,0.000000,11.611988


In [219]:
def getUserDistCorrelation(distance_matrix, name):
    df2_user=distance_matrix.loc[name,:]
    df2_scaled_user = 1-MinMaxScaler().fit_transform(pd.DataFrame(df2_user))
    return df2_scaled_user

In [220]:
test_user_distance = getUserDistCorrelation(test_distance_matrix, 'Amoli Veer')
test_user_distance

array([[1.        ],
       [0.9944079 ],
       [0.99463664],
       ...,
       [0.98763991],
       [0.99949552],
       [0.99060104]])

In [221]:
def getFinalRecommendatonMatrix(Rec1, Rec2 , name):
    df_recommedation = pd.DataFrame(Rec1)
    df_recommedation['Haversine'] = Rec2
    df_recommedation.rename(columns = {name:'Cosine Similarity'}, inplace = True)
    return df_recommedation

In [222]:
test_final_rec = getFinalRecommendatonMatrix(test_user_corr,test_user_distance,'Amoli Veer' )

In [223]:
test_final_rec

,Cosine Similarity,Haversine
Name,,
Amoli Veer,1.000000,1.000000
Kaia Subramanian,0.576278,0.994408
Rushil Vish,0.764181,0.994637
Atiksh Manohar,0.361788,0.993220
Ved Kapoor,0.475178,0.994956
...,...,...
Neysa Rai,0.647972,0.994111
Raunak Kapoor,0.922747,0.987640
Taara Veer,0.931432,0.987640


In [224]:
def prioritySetter(df_recommedation,df, priority1, priority2):
    df_recommedation['Score'] = priority1* df_recommedation['Cosine Similarity'] + priority2* df_recommedation['Haversine']
    Hybrid_Recommendation=df_recommedation.sort_values(['Score'], ascending=False)
    Hybrid_rec = Hybrid_Recommendation.index.to_list()
    final_rec = df.set_index('Name').loc[Hybrid_rec[:]].reset_index(inplace=False)
    return final_rec

In [227]:
prioritySetter(test_final_rec, df, 0.1,0.9).head(10)

,Name,Gender,DOB,Name Of Area,Pincode,Latitude,Longitude,Flexibility,Body Composition,Composition Percentage,Interactiveness,Workout Frequency,Workout Mode Preferred,is_sports_enthusiast,like_to_be_mentored,like_to_mentor,fitness_goal_to_be achieved,addons in session,Height_in_cms,Age
0,Amoli Veer,Female,1982-03-31,Nana Peth S.O,411002,18.51009,73.85672,4,Essential Fat,12,4,Only Weekends,Outdoor,No,3,3,All,Meditation,166,39
1,Alex Pawan,Male,1989-10-24,Narayan Peth S.O,411030,18.51553,73.85230,4,Essential Fat,6,1,Only Weekends,Outdoor,No,3,5,Build muscle,None,162,31
2,Saisha Nirmal,Female,2000-10-15,Nana Peth S.O,411002,18.51009,73.85672,1,Essential Fat,12,1,Only Weekends,Indoor,No,5,5,All,Both,165,20
3,Aaradhya Soni,Female,2000-05-15,Lokmanyanagar S.O,411030,18.50825,73.84405,3,Fitness,22,5,Only Weekends,Indoor,Yes,2,5,All,Meditation,167,21
4,Rishaan Kishore,Male,1991-03-06,Lokmanyanagar S.O,411030,18.50825,73.84405,4,Essential Fat,6,1,Only Weekends,Outdoor,Maybe,2,4,All,None,176,30
5,Naitee Chauhan,Female,2000-11-27,Rasta Peth S.O,411011,18.52420,73.85780,1,Essential Fat,12,1,Only Weekends,Outdoor,Yes,1,4,All,None,172,20
6,Rohan Tyagi,Male,1995-04-13,Lokmanyanagar S.O,411030,18.50825,73.84405,4,Essential Fat,6,1,Only Weekends,Indoor,No,2,1,Build muscle,Meditation,168,26
7,Nehrika Malhotra,Female,1992-02-20,Bajirao Road S.O,411002,18.50995,73.85375,5,Essential Fat,12,4,Only Weekends,Both,No,5,1,All,Meditation,169,29
8,Kashvi Chande,Female,1998-08-21,Market Yard S.O (Pune),411037,18.48193,73.87425,2,Fitness,22,2,Only Weekends,Outdoor,No,4,3,All,Both,169,22
9,Asmee Patel,Female,1981-04-22,Pune New Bazar S.O,411001,18.53446,73.88304,2,Athletes,19,3,Only Weekends,Outdoor,No,5,4,Build muscle,Both,168,40


In [208]:
Name_list = df.Name.to_list()
name_list = widgets.Dropdown(
    options=Name_list,
    description='Names:',
)
Priority1_slider = widgets.FloatSlider(
    value=0.5,
    min=0,
    max=1.0,
    step=0.01,
    description='Personality Priority:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.2f',
)

display(name_list,Priority1_slider)
button = widgets.Button(
    description='Run',
    disabled=False,
    button_style='success', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Click me',
    icon='check' # (FontAwesome names without the `fa-` prefix)
    
#     value=0
)
display(button)

def on_button_clicked(b):
    name = name_list.value
    priority1 = Priority1_slider.value
    priority2 = 1- priority1
    print('Personality Priority: {} Percent & Location Priority: {} Percent are set!'.format(priority1*100, priority2*100))
    df = pd.read_csv("Final Dataset before Normalisation.csv", index_col=[0])
    df1,df2 = get_both_datasets_for_normalisation(df)
    names = ['Flexibility','Composition Percentage','Pincode', 'Interactiveness', 'like_to_be_mentored', 'like_to_mentor', 'Height_in_cms', 'Age']
    df1 = get_df1_normalised(df1,names)
    df1_scaled = df1.copy()
    df1_similarity= getSimilarityMatrix(df1_scaled)
    user_corr = getUserCorrelation(df1_similarity, name)
    df2 = addLatLongRad(df2)
    distance_matrix = getDistanceMatrix(df2)
    user_distance = getUserDistCorrelation(test_distance_matrix, name)
    final_rec = getFinalRecommendatonMatrix(user_corr,user_distance,name)
    Recommendations= prioritySetter(final_rec, df, priority1,priority2)
    display(Recommendations.head(10))

button.on_click(on_button_clicked)


Dropdown(description='Names:', options=('Amoli Veer', 'Kaia Subramanian', 'Rushil Vish', 'Atiksh Manohar', 'Ve…

FloatSlider(value=0.5, continuous_update=False, description='Personality Priority:', max=1.0, step=0.01)

Button(button_style='success', description='Run', icon='check', style=ButtonStyle(), tooltip='Click me')

Personality Priority: 50.0 Percent & Location Priority: 50.0 Percent are set!


,Name,Gender,DOB,Name Of Area,Pincode,Latitude,Longitude,Flexibility,Body Composition,Composition Percentage,Interactiveness,Workout Frequency,Workout Mode Preferred,is_sports_enthusiast,like_to_be_mentored,like_to_mentor,fitness_goal_to_be achieved,addons in session,Height_in_cms,Age
0,Anushka Neelam,Female,1982-02-01,Model Colony S.O,411016,18.52906,73.83811,4,Acceptable,28,4,Any Day of Week,Indoor,No,2,3,Improve athletic skills,Yoga,171,39
1,Hiya Punj,Female,1995-02-15,Mundhva B.O,411036,18.53135,73.91478,2,Acceptable,28,4,Any Day of Week,Indoor,No,2,3,Improve athletic skills,Yoga,177,26
2,Ishana Nigam,Female,1984-07-05,Aundh T.S. S.O,411007,18.55144,73.82197,4,Acceptable,28,5,Alternate Days,Indoor,Yes,1,4,Improve athletic skills,Yoga,174,36
3,Keya Veer,Female,1986-07-04,Dapodi Bazar S.O,411012,18.57995,73.82783,2,Acceptable,28,5,Daily,Outdoor,Yes,1,5,Improve endurance/conditioning,Yoga,162,34
4,Mayra Vish,Female,1986-09-15,Ganeshkhind S.O,411007,18.55144,73.82197,5,Acceptable,28,1,Monday To Friday,Outdoor,Yes,1,5,Improve endurance/conditioning,Yoga,172,34
5,Anika Subram,Female,1987-06-14,Guruwar Peth S.O,411042,18.50547,73.85951,1,Acceptable,28,5,Any Day of Week,Outdoor,Maybe,1,4,Improve athletic skills,Yoga,171,33
6,Zara Mati,Female,2003-04-15,Katraj S.O,411046,18.45938,73.86667,4,Acceptable,28,5,Alternate Days,Indoor,No,5,3,Improve athletic skills,Yoga,169,18
7,Nimit Subramanian,Male,1992-03-04,Bhosari I.E. S.O,411026,18.55284,73.95387,5,Athletes,10,4,Any Day of Week,Indoor,No,2,5,Improve athletic skills,Yoga,170,29
8,Kaia Vijaya,Female,1990-04-24,Bhosarigoan S.O,411039,18.55284,73.95387,3,Acceptable,28,2,Daily,Indoor,No,2,2,Build muscle,Yoga,174,31
9,Joseph Mittal,Male,2002-02-22,Bhusari Colony S.O,411038,18.50956,73.79646,1,Acceptable,22,3,Daily,Indoor,No,2,4,Improve endurance/conditioning,Yoga,174,19


Personality Priority: 91.0 Percent & Location Priority: 8.999999999999996 Percent are set!


,Name,Gender,DOB,Name Of Area,Pincode,Latitude,Longitude,Flexibility,Body Composition,Composition Percentage,Interactiveness,Workout Frequency,Workout Mode Preferred,is_sports_enthusiast,like_to_be_mentored,like_to_mentor,fitness_goal_to_be achieved,addons in session,Height_in_cms,Age
0,Anushka Neelam,Female,1982-02-01,Model Colony S.O,411016,18.52906,73.83811,4,Acceptable,28,4,Any Day of Week,Indoor,No,2,3,Improve athletic skills,Yoga,171,39
1,Hiya Punj,Female,1995-02-15,Mundhva B.O,411036,18.53135,73.91478,2,Acceptable,28,4,Any Day of Week,Indoor,No,2,3,Improve athletic skills,Yoga,177,26
2,Keya Veer,Female,1986-07-04,Dapodi Bazar S.O,411012,18.57995,73.82783,2,Acceptable,28,5,Daily,Outdoor,Yes,1,5,Improve endurance/conditioning,Yoga,162,34
3,Ishana Nigam,Female,1984-07-05,Aundh T.S. S.O,411007,18.55144,73.82197,4,Acceptable,28,5,Alternate Days,Indoor,Yes,1,4,Improve athletic skills,Yoga,174,36
4,Nimit Subramanian,Male,1992-03-04,Bhosari I.E. S.O,411026,18.55284,73.95387,5,Athletes,10,4,Any Day of Week,Indoor,No,2,5,Improve athletic skills,Yoga,170,29
5,Kaia Vijaya,Female,1990-04-24,Bhosarigoan S.O,411039,18.55284,73.95387,3,Acceptable,28,2,Daily,Indoor,No,2,2,Build muscle,Yoga,174,31
6,Zara Mati,Female,2003-04-15,Katraj S.O,411046,18.45938,73.86667,4,Acceptable,28,5,Alternate Days,Indoor,No,5,3,Improve athletic skills,Yoga,169,18
7,Mayra Vish,Female,1986-09-15,Ganeshkhind S.O,411007,18.55144,73.82197,5,Acceptable,28,1,Monday To Friday,Outdoor,Yes,1,5,Improve endurance/conditioning,Yoga,172,34
8,Anika Subram,Female,1987-06-14,Guruwar Peth S.O,411042,18.50547,73.85951,1,Acceptable,28,5,Any Day of Week,Outdoor,Maybe,1,4,Improve athletic skills,Yoga,171,33
9,Rishaan Sharma,Male,1981-12-06,Vishrantwadi S.O,411015,18.60226,73.89806,4,Athletes,10,3,Daily,Outdoor,No,5,5,Improve endurance/conditioning,Yoga,178,39


In [189]:
df['addons in session'].value_counts()

None          361
Yoga          349
Both          340
Meditation    337
Name: addons in session, dtype: int64

In [228]:
import requests

URL = "https://geocode.search.hereapi.com/v1/geocode"
location = input("Enter the location here: ") #taking user input
api_key = 'sO0hb7sykYZPwb-O8Y2PBv3w7bpy69IKrPmbPBxIRcU' # Acquire from developer.here.com
PARAMS = {'apikey':api_key,'q':location} 

# sending get request and saving the response as response object 
r = requests.get(url = URL, params = PARAMS) 
data = r.json()
print(data)
latitude = data['items'][0]['position']['lat']
longitude = data['items'][0]['position']['lng']
postal_code = data['items']

Enter the location here: Undri
{'items': [{'title': 'Undri, Pune, Maharashtra, India', 'id': 'here:cm:namedplace:28105027', 'resultType': 'locality', 'localityType': 'district', 'address': {'label': 'Undri, Pune, Maharashtra, India', 'countryCode': 'IND', 'countryName': 'India', 'stateCode': 'MH', 'state': 'Maharashtra', 'county': 'Pune', 'city': 'Pune', 'district': 'Undri', 'postalCode': '411028'}, 'position': {'lat': 18.45427, 'lng': 73.91788}, 'mapView': {'west': 73.8999, 'south': 18.43403, 'east': 73.92575, 'north': 18.46493}, 'scoring': {'queryScore': 1.0, 'fieldScore': {'district': 1.0}}}, {'title': 'Undri, Mukhed, Maharashtra, India', 'id': 'here:cm:poi:1001416458', 'resultType': 'locality', 'localityType': 'district', 'address': {'label': 'Undri, Mukhed, Maharashtra, India', 'countryCode': 'IND', 'countryName': 'India', 'stateCode': 'MH', 'state': 'Maharashtra', 'county': 'Nanded', 'city': 'Mukhed', 'district': 'Undri'}, 'position': {'lat': 18.59543, 'lng': 77.48493}, 'mapView'

In [134]:
postal_code = data['items'][0]['address']['postalCode']

In [135]:
postal_code

'411058'

In [204]:

name = widgets.Text(
value='',
placeholder='Type something',
description='Name:',
disabled=False
    )
gender = widgets.RadioButtons(
    options=['Male', 'Female'],
    description='Gender',
    disabled=False
    )
date = widgets.DatePicker(
    description='date of birth',
    disabled=False
    )
name_of_area = widgets.Text(
    value='',
    placeholder='Type something',
    description='Locality:',
    disabled=False
    )
URL = "https://geocode.search.hereapi.com/v1/geocode"
api_key = 'sO0hb7sykYZPwb-O8Y2PBv3w7bpy69IKrPmbPBxIRcU' # Acquire from developer.here.com
PARAMS = {'apikey':api_key,'q':location} 

# sending get request and saving the response as response object 
r = requests.get(url = URL, params = PARAMS) 
data = r.json()
latitude = data['items'][0]['position']['lat']
longitude = data['items'][0]['position']['lng']
postal_code = data['items'][0]['address']['postalCode']
    
flexibility = widgets.IntSlider(
    value=2,
    min=1,
    max=5,
    step=1,
    description='Flexibilty',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d'
    )
body_composition = widgets.Dropdown(
    options=['Acceptable', 'Athletes', 'Obese', 'Essential Fat', 'Fitness'],
    value='Acceptable',
    description='Body Composition:',
    disabled=False,
    )
composition_percentage = widgets.IntSlider(
    min=1,
    max=100,
    step=1,
    description='Composition Percentage:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d'
    )
interactiveness = widgets.IntSlider(
    value=2,min=1, max=5,step=1, description='Interactiveness :', disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d'
    )
workout_frequency = widgets.Dropdown(
    options=['Monday To Friday', '3 days of week', 'Alternate Days', 'Any Day of Week', 'Daily','Only Weekends'],
    description='Workout Frequency:',
    disabled=False,
    )
workout_mode = widgets.Dropdown(
    options=['Outdoor', 'Indoor', 'Both'],
    description='Workout Mode Preferred:',
    disabled=False,
    )
is_sport_enthusiast = widgets.RadioButtons(
    options=['Yes', 'No', 'Maybe'],
    description='Are You Sports Enthusiast',
    disabled=False
    )
like_to_be_mentored = widgets.IntSlider(
    min=1, max=5,step=1, description='Do You Like to be mentored? :', disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d'
    )
like_to_mentor = widgets.IntSlider(
    min=1, max=5,step=1, description='Do You Like to mentor? :', disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d'
    )
fitness_Goal = widgets.RadioButtons(
    options=['All', 'Improve endurance/conditioning', 'Improve athletic skills', 'Lose fat', 'Mesomorph'],
    description='Fitness Goal?',
    disabled=False
    )
addon_session = widgets.RadioButtons(
    options=['Meditation', 'Yoga', 'Both', 'None'],
    description='Addon Session required?',
    disabled=False
    )
height = widgets.IntSlider(
    value=160,min=160, max=180,step=1, description='Height in centimeters:', disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d'
    )
display(name, gender,date, name_of_area,flexibility,body_composition,composition_percentage,interactiveness,workout_frequency, workout_mode, is_sport_enthusiast,like_to_be_mentored,like_to_mentor, fitness_Goal,addon_session,height)
    
enter_data = widgets.Button(
    description='Submit',
    disabled=False,
    button_style='success', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Click me',
    icon='check' # (FontAwesome names without the `fa-` prefix)
    
#     value=0
)
display(enter_data)


Text(value='', description='Name:', placeholder='Type something')

RadioButtons(description='Gender', options=('Male', 'Female'), value='Male')

DatePicker(value=None, description='date of birth')

Text(value='', description='Locality:', placeholder='Type something')

IntSlider(value=2, continuous_update=False, description='Flexibilty', max=5, min=1)

...

IntSlider(value=1, continuous_update=False, description='Do You Like to mentor? :', max=5, min=1)

RadioButtons(description='Fitness Goal?', options=('All', 'Improve endurance/conditioning', 'Improve athletic …

RadioButtons(description='Addon Session required?', options=('Meditation', 'Yoga', 'Both', 'None'), value='Med…

IntSlider(value=160, continuous_update=False, description='Height in centimeters:', max=180, min=160)

Button(button_style='success', description='Submit', icon='check', style=ButtonStyle(), tooltip='Click me')

In [206]:
longitude

73.80213

In [201]:
location = name_of_area.value
print(location )

NameError: name 'name_of_area' is not defined

In [144]:
g.value

'Female'

In [211]:
df

,Name,Gender,DOB,Name Of Area,Pincode,Latitude,Longitude,Flexibility,Body Composition,Composition Percentage,Interactiveness,Workout Frequency,Workout Mode Preferred,is_sports_enthusiast,like_to_be_mentored,like_to_mentor,fitness_goal_to_be achieved,addons in session,Height_in_cms,Age
0,Amoli Veer,Female,1982-03-31,Nana Peth S.O,411002,18.51009,73.85672,4,Essential Fat,12,4,Only Weekends,Outdoor,No,3,3,All,Meditation,166,39
1,Kaia Subramanian,Female,1994-06-09,Mundhva AV S.O,411036,18.53135,73.91478,5,Fitness,22,5,Alternate Days,Outdoor,Maybe,1,3,Improve endurance/conditioning,Meditation,163,26
2,Rushil Vish,Male,1999-02-07,Yerwada S.O,411006,18.56318,73.87742,3,Acceptable,22,3,Monday To Friday,Both,Yes,4,5,Improve athletic skills,Meditation,171,22
3,Atiksh Manohar,Male,1987-07-01,Dapodi S.O,411012,18.57749,73.83138,5,Athletes,10,3,Alternate Days,Both,Maybe,3,4,Lose fat,Meditation,173,33
4,Ved Kapoor,Male,1988-01-28,Hadapsar S.O,411028,18.50453,73.91255,5,Essential Fat,6,3,3 days of week,Indoor,No,1,1,Build muscle,None,161,33
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1382,Neysa Rai,Female,1989-12-10,Mohamadwadi S.O,411060,18.47335,73.90957,2,Fitness,22,4,Any Day of Week,Both,Maybe,2,4,Improve athletic skills,Yoga,170,31
1383,Raunak Kapoor,Male,1984-08-12,Masulkar Colony S.O,411018,18.63374,73.81290,3,Fitness,15,1,Monday To Friday,Outdoor,Yes,2,5,All,None,161,36
1384,Taara Veer,Female,1995-06-06,Masulkar Colony S.O,411018,18.63374,73.81290,5,Fitness,22,1,Only Weekends,Indoor,No,5,4,Build muscle,Both,166,25
1385,Sarthak Rai,Male,1997-09-03,Guruwar Peth S.O,411042,18.50547,73.85951,5,Fitness,15,5,Monday To Friday,Indoor,Maybe,3,2,Improve athletic skills,Yoga,177,23
